In [48]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
keys = pd.read_csv('your keys here')
consumer_key = keys['keys'][0]
consumer_secret = keys['keys'][1]
access_token = keys['keys'][2]
access_secret = keys['keys'][3]
keys

In [50]:
authenticate = tweepy.OAuthHandler(consumer_key, consumer_secret) 
authenticate.set_access_token(access_token, access_secret) 
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [51]:
def getUserPosts(user, number):
    timeline = api.user_timeline(screen_name = user, count = number, lang ="en", tweet_mode = "extended")
    arr = []
    j=1
    for i in timeline[:number]:
        arr.append(i.full_text)
        j += 1
    return arr
def getHashtagPosts(hashtag, number, date = None):
    timeline = list(tweepy.Cursor(api.search,q = hashtag,count = number, lang = "en", since = date).items())
    arr = []
    j=1
    for i in timeline[:number]:
        arr.append(i.full_text)
        j += 1
    print(arr)
    return arr
def clean(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) 
    text = re.sub('RT[\s]+', '', text) 
    text = re.sub('https?:\/\/\S+', '', text)
 
    return text

In [52]:
def getSubjectivity(df, col):
    arr = []
    for i in df[col]:
        arr.append(TextBlob(i).sentiment.polarity)
    return arr

def getPolarity(df, col):
    arr = []
    for i in df[col]:
        arr.append(TextBlob(i).sentiment.polarity)
    return arr


In [53]:
def analyze(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [54]:
def getSentiment(col, target, num, date = None):
    if target[0] == '#':
        arr = getHashtagPosts(target, num, date)
        frame = pd.DataFrame(arr, columns = ['Tweets'])
        cleaned = frame[col].apply(clean)
        cleaned  = cleaned.to_frame()
        cleaned['Subjectivity'] = getSubjectivity(cleaned, col)
        cleaned['Polarity'] = getPolarity(cleaned, col)
        cleaned['Outcome'] = cleaned['Polarity'].apply(analyze)
    else:
        arr = getUserPosts(target, num)
        frame = pd.DataFrame(arr, columns = ['Tweets'])
        cleaned = frame[col].apply(clean)
        cleaned  = cleaned.to_frame()
        cleaned['Subjectivity'] = getSubjectivity(cleaned, col)
        cleaned['Polarity'] = getPolarity(cleaned, col)
        cleaned['Outcome'] = cleaned['Polarity'].apply(analyze)
    return cleaned

In [55]:
def sortdf(dataframe, sorter, order):
    return df.sort_values(by=['Polarity'], ascending = order )

In [57]:
def makeWordPlot(df, col):
    words = ' '.join([t for t in df[col]])
    wordCloud = WordCloud(width=500, height=300, random_state=21, max_font_size=110).generate(words)
    plt.imshow(wordCloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()